### Creating text dataset to match the structured dataset before cleaning

- Current data uniquely identified by PSID, ReferralStartDate and AssessmentType
- Previous data uniquely identified by PSID, ReferralStartDate and AssessmentType dummies
- Keys: document has ActualStartDate within ReferralStartDate - ReferralCloseDate period (minus 1 day)
- It appears sometimes the document ActualStartDate is the day before

- Population: rq1: at the point of referral (only keep Contact and Referral form)
- Population: rq2: open cases (documents with ActualStartDate within 2 weeks of ReferralDatetime)
- Aggregate text by DocumentName

In [ ]:
import sys
sys.path
sys.path.append('C:\\Program Files\\Python37\\Lib\\site-packages')
sys.path.append('C:\Program Files\Python37')

In [ ]:
import os
os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]

### Bring all the data and store in a pickled dictionary

In [ ]:
assert 1==2
# Takes a while to run
# Use "Created/all_text_dict.pkl"

In [ ]:
import glob
import pickle
list_of_text_files = glob.glob("My Documents*.xlsx")

pseudo_id_misspellings = ['PseudoId', 'PsuedoID', 'PesudoID','PsueodID']
pseudo_id_rename = dict(zip(pseudo_id_misspellings,['PseudoID'] *4))

df_list = []
for file in list_of_text_files:
    print(file)
    try:
        df = pd.read_excel(file)
        print(df.shape)
    except(UnicodeDecodeError):
        # Some datafiles are encoded differently
        df = pd.read_excel(file,  encoding = 'latin')
        print(df.shape)
    print(df.shape)
    # PseudoID spelt slightly differently
    if set(pseudo_id_misspellings).intersection(set(df.columns)):
        df.rename(columns =pseudo_id_rename, inplace = True)
    # Some columns are completely empty as there's no answer of that datatype
    df.dropna(axis = 1, how = 'all', inplace = True)
    df.dropna(axis = 0, how = 'all', inplace = True)
    df.dropna(subset = ['ActualStartDate', 'PSID'], how = 'any', inplace = True)
    try:
        df.drop(columns = ['Question', 'QuestionType'], inplace = True)
    except:
        continue
    # Standardise document names
    df['DocumentName'] = df['DocumentName'].map(lambda x: str(x).strip("(c) "))
    df['DocumentName'] = df['DocumentName'].map(lambda x: str(x).strip(" v3.1"))
    # Make start date date time object
    df['ActualStartDate'] = pd.to_datetime(df['ActualStartDate'], format = "%m/%d/%Y %H:%M")
    df['ActualStartDate'] = pd.to_datetime(df['ActualStartDate'].map(lambda x: x.strftime('%Y-%m-%d')))
    print(df.shape)
    # Harmonising column names and dropping duplicate columns
    pseudoid = df.columns[len(df.columns)-1]
    print(pseudoid)
    df.rename(columns = {pseudoid: "PseudoID_keep"}, inplace = True)
    df.drop(columns = df.columns[(len(df.columns) - 3)], inplace = True)
    print(df.columns)
    df_list.append(df)
    
# MyDocuments04806 - row 279/ 280 corrupted; lose c.150 observations
## Create dictionary of dataframes with key as file name
df_dict = dict(zip(list_of_text_files, df_list))
df_dict2 = dict(df_dict)

with open("Created/all_text_dict.pkl", "wb") as handle:
    pickle.dump(df_dict2, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Combined dataframes in pickled dictionary into a single dataset

In [ ]:
assert 1==2

In [ ]:
# Bring in all text data stored as a dictionary
# Keys = document name; values = dataframe
import os
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename = open("Created/all_text_dict.pkl", "rb")
df_dict2 = pickle.load(filename)
df_dict = dict(df_dict2)

In [ ]:
# Question list is meta-data matching the question ID to the question and the form it's on
# At the moment just used for deciding which files to drop
import pandas as pd
question_list = pd.read_excel(".\\Other\\QuestionListfull.xlsx")
question_list['Question ID'] = question_list['Question ID'].map(lambda x: x.strip("CHILD "))
question_list['Document Name (All)'] = question_list['Document Name (All)'].map(lambda x: x.strip("(c) "))
question_list.head()

# Link to structured data using contact and referral documents - identify contact and referral documents
question_IDs_contact_referral = list(question_list.loc[question_list['Document Name (All)'] == 'Contact and Referral Form', 'Question ID'])

In [ ]:
# Check what questions the documents cover
import glob
import re
list_of_text_files = glob.glob("My Documents*.xlsx")
list_of_text_files_num = [re.sub("My Documents", "", file) for file in list_of_text_files]
list_of_text_files_num = [re.sub(".xlsx", "", file) for file in list_of_text_files_num]

question_id_dict = dict(zip(list_of_text_files, list(question_list.loc[question_list['Question ID'].isin(list_of_text_files_num),'Section Question'])))

# Drop documents with prescient columns
# Outcome of this assessment - 04778, 04809, 04813
# 09996, 09998 - Duplicate of 'My Documents08728.xlsx' (child risk factors at contact)
# 09997 - Duplicate of 'My Documents08648.xlsx' (parental risk factors at contact)
# 07754 - Parent / carers name
documents_to_exclude = (['My Documents04778.xlsx', 'My Documents04809.xlsx', 
                         'My Documents04813.xlsx', 'My Documents09996.xlsx', 'My Documents09997.xlsx',
                        'My Documents09998.xlsx', 'My Documents07754.xlsx'])

for k in documents_to_exclude:
    question_id_dict.pop(k, None)
    df_dict.pop(k, None)
    df_dict2.pop(k, None)

In [ ]:
# Identify dfs with multiple answer columns
multiple_answer_dfs = {}
for df_name, df in df_dict2.items():
    print(df.columns)
    answer_cols = [col for col in df.columns if 'MultipleChoiceAnswer' in col]
    if answer_cols != []:
        print(df_name)
        multiple_answer_dfs[df_name] = df 
        del df_dict[df_name]
print(len(df_dict.keys())) 
print(len(multiple_answer_dfs.keys()))
print(len(df_dict2.keys())) 

In [ ]:
## Multiple answer questions have multiple rows per child / actual start date 
## if multiple answers are given, change from long to wide 
for df_name, df in multiple_answer_dfs.items():
    print(df.shape)
    df_de_dup = df.drop_duplicates(subset = ['PSID', 'ActualStartDate', 'MultipleChoiceAnswer'], keep = 'first')
    print(df_de_dup.shape)
    # Turn into dummies (=> multiple rows, multiple columns)
    df_de_dup_multiple = pd.get_dummies(df_de_dup['MultipleChoiceAnswer'], prefix = 'Answer_{}'.format(df_de_dup['QuestionMarker'][0]))
    df_de_dup = pd.concat([df_de_dup, df_de_dup_multiple], axis = 1)
    df_de_dup.drop(columns = 'MultipleChoiceAnswer', inplace = True)
    # Collapse to one row per child / actual start date
    df_de_dup_max = df_de_dup.groupby(['PSID', 'ActualStartDate'])[df_de_dup_multiple.columns].max()
    df_de_dup_max.reset_index(drop = False, inplace = True)
    # Merge back into main dataframe
    df_de_dup_wide = df_de_dup[['ActualStartDate', 'DocumentName', 'SectionName', 'QuestionMarker',
           'PSID']].merge(df_de_dup_max, how = 'left', on = ['PSID', 'ActualStartDate'])
    print(df_de_dup_wide.shape)
    df_dict2[df_name] = df_de_dup_wide

In [ ]:
# Deduplicate after having dealt with multiple answer columns
for df_name, df in df_dict2.items():
    print(df_name)
    print(df.shape)
    answer = [col for col in df.columns if 'Answer' in col]
    print(answer)
    if answer:
        cols_de_dup = answer +  ['PSID', 'ActualStartDate']
        df_de_dup = df.drop_duplicates(subset = cols_de_dup, keep = 'first')
        print(df.shape)
        df_dict2[df_name] = df_de_dup

In [ ]:
## Concatenate text in the same document for the same child on the same ActualStartDate
for df_name, df in df_dict2.items():
    print(df_name)
    print(df.shape)
    answers = [col for col in df.columns if 'Answer' in col]
    for answer in answers:
        if "Answer_CHILD" not in answer:
            print(answer)
            df[answer] = df[answer].fillna('')
            print("Number missing: ", df[answer].isna().sum())
            df = df.loc[:,~df.columns.duplicated()]
            print(df['DocumentName'].value_counts())
            df_answer_concat = df.groupby(['PSID', 'ActualStartDate', 'DocumentName'], as_index = False)[answer].agg(' '.join)
            print(df_answer_concat.shape)
            df_pivoted = df_answer_concat.set_index(['PSID', 'ActualStartDate']).pivot(columns = 'DocumentName')[answer].reset_index().rename_axis(None, axis=1)
            print(df_pivoted.shape)
            df_dict2[df_name] = df_pivoted

In [ ]:
# Renaming columns in preparation for merging
# Otherwise end up with _x and _y repeat columns

cols_to_combine = (['Child Social Work Assessment for Review Child Protection Conference',
            'Child Social Work Assessment to Initial Child Protection Conference',
            'Child Social Work Assessment',
            'Contact and Referral Form'])

for df_name, df in df_dict2.items():
    suf = re.findall('[0-9]+', df_name)
    print(suf[0])
    #df.columns = df.columns.map(lambda x: x+suf[0] if x != 'PSID' and x != 'ActualStartDate' else x)
    df.columns = df.columns.map(lambda x: x+suf[0] if x in cols_to_combine else x)
    print(df.columns)

In [ ]:
## Can now merge together text dataframes
from functools import reduce
import pickle

df_all_text = reduce(lambda x, y: pd.merge(x, y, how = 'outer', on = ['ActualStartDate', 'PSID']), df_dict2.values())

with open("Created\\df_all_text.pkl", "wb") as handle:
    pickle.dump(df_all_text, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Combine text from different forms

In [ ]:
assert 1==2
# Takes a while to run
# Use "Created\\df_combined_all_text.pkl"

In [ ]:
import os
import pandas as pd
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename = open("Created\\df_all_text.pkl", "rb")
df_all = pickle.load(filename)

repeat_cols_y = [col for col in df_all.columns if '_y' in col]
repeat_cols_x = [col for col in df_all.columns if '_x' in col]

repeat_cols = repeat_cols_y + repeat_cols_x

df_all.drop(columns = repeat_cols, inplace = True)

cols_to_combine = (['Child Social Work Assessment for Review Child Protection Conference',
            'Child Social Work Assessment to Initial Child Protection Conference',
            'Child Social Work Assessment',
            'Contact and Referral Form'])

relevant_cols_all = []
for column in cols_to_combine: 
    print("Column: ", column)
    relevant_cols = [col for col in df_all.columns if column in col]
    #print("Relevant columns: ", relevant_cols)
    # Select relevant columns
    # Fillna crashes the computer when trying to do all at once
    relevant_cols_df = df_all[relevant_cols].fillna('')
    # Check that filling na worked - yes
    #print(relevant_cols_df.isna().sum())
    df_all[column + '_text'] = ''
    for idx in df_all.index:
        # Set to get rid of duplicates
        list_text = list(set([t for t in relevant_cols_df.loc[idx,]]))
        # Join together text 
        df_all.loc[idx, column + '_text'] = ' '.join(list_text)
        # Check length of new combined document == sum of documents (Yes plus a few \n)
        #print("Sum of length of all relevant cells: ", sum([len(df_all.loc[idx, col]) for col in relevant_cols if type(df_all.loc[idx, col]) != float]))
        #if type(df_all.loc[idx,  column + '_text']) != float:
            #print("Length of new cell: ", len(df_all.loc[idx, column + '_text']))
        if (idx % 1000 ==0) & (idx != 0):
            print("Idx: ", idx)
            with open("Created\\Combined\\df_all_combined_{}_{}_{}.pkl".format(column, (idx-1000), idx), "wb") as handle:
                pickle.dump(df_all.loc[(idx-1000):idx,['PSID', 'ActualStartDate',column + '_text']], handle, protocol = pickle.HIGHEST_PROTOCOL)
        if idx == (df_all.shape[0]-1):
            print("Idx: ", idx)
            with open("Created\\Combined\\df_all_combined_{}_{}_{}.pkl".format(column, (idx-(df_all.shape % 1000)), idx), "wb") as handle:
                pickle.dump(df_all.loc[(idx-(df_all.shape % 1000)):idx,['PSID', 'ActualStartDate',column + '_text']], handle, protocol = pickle.HIGHEST_PROTOCOL)            
    relevant_cols_all.extend(relevant_cols)


In [ ]:
## Bring concatenated data back in
import glob
import os
import pandas as pd
import pickle
import re

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
list_of_text_files = glob.glob("Created\\Combined\\*.pkl")

df_combined_files_dict = {}
for file in list_of_text_files:
    filename = open(file, "rb")
    df_combined = pickle.load(filename)
    df_combined_files_dict[file] = df_combined

# Concatenate dataframes with the same columns (axis = 0)
# Create a list of dataframes with different columns for merging
list_of_text_files_to_concat = list(set([re.sub('[0-9+]', '', file) for file in list_of_text_files]))
list_of_text_files_to_concat = list(set([re.sub('_.pkl', '', file) for file in list_of_text_files_to_concat]))

df_combined_list_all = []
for file in list_of_text_files_to_concat:
    print(file)
    relevant_files = [f for f in list_of_text_files if file in f]
    #print(relevant_files)
    df_combined_list = []
    for f in relevant_files:
        df_combined_list.append(df_combined_files_dict[f])
    df_combined = pd.concat(df_combined_list, axis = 0)
    print(df_combined.shape)
    df_combined_list_all.append(df_combined)

In [ ]:
# Merge dataframes with different text columns
from functools import reduce
df_combined_all_text = reduce(lambda left, right: pd.merge(left, right, on = ['PSID', 'ActualStartDate'], how = 'outer'), df_combined_list_all)

# Look at number of unique entries
for col in df_combined_all_text.columns:
    print(col)
    print(df_combined_all_text[col].nunique())

with open("Created\\df_combined_all_text.pkl", "wb") as handle:
    pickle.dump(df_combined_all_text, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
df_combined_all_text.loc[df_combined_all_text['Child Social Work Assessment for Review Child Protection Conference_text'] != '', ['ActualStartDate', 'Child Social Work Assessment for Review Child Protection Conference_text']].isna().sum()

### Combine with rq1data

In [ ]:
assert 1==2

In [ ]:
import os
import pandas as pd
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename = open("Created\\df_combined_all_text.pkl", "rb")
df_combined_all_text = pickle.load(filename)
print(df_combined_all_text.shape)

rq1data = pd.read_csv("..\\Updated Structured Data\\Created\\rq1data_w_previous.csv", index_col = 0)
print(rq1data.shape)
#print(rq1data.index)
#print(rq1data.columns)

In [ ]:
# Merge in just structured data required to select relevant text data
# Don't want all structured data too yet as we'll need to have just text data for extracting text features

rq1data_text = rq1data[['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate']].merge(df_combined_all_text, how = 'outer', on = 'PSID')
rq1data_text[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text']] = (rq1data_text[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
                                                               'Child Social Work Assessment to Initial Child Protection Conference_text',
                                                               'Child Social Work Assessment_text', 'Contact and Referral Form_text']].fillna(''))
(rq1data_text.rename(columns = {'ActualStartDate_x':'ActualStartDate_str',
                                 'ActualStartDate_y':'ActualStartDate_txt'},
                                inplace = True))

rq1data_text['ActualStartDate_str'] = pd.to_datetime(rq1data_text['ActualStartDate_str'], format = "%Y-%m-%d")
rq1data_text['ReferralDatetime'] = pd.to_datetime(rq1data_text['ReferralDatetime'], format = "%Y-%m-%d")
rq1data_text['ReferralCloseDate'] = pd.to_datetime(rq1data_text['ReferralCloseDate'], format = "%Y-%m-%d")

In [ ]:
rq1data_text.loc[rq1data_text['Child Social Work Assessment for Review Child Protection Conference_text'] != '', ['ActualStartDate','ActualStartDate_str', 'Child Social Work Assessment for Review Child Protection Conference_text']].shape

In [ ]:
rq1data_text.loc[rq1data_text['Child Social Work Assessment for Review Child Protection Conference_text'] != '', ['ActualStartDate','ActualStartDate_str', 'Child Social Work Assessment for Review Child Protection Conference_text']].isna().sum()

In [ ]:
# Check merges
'''
rq1data_text['ActualStartDateMatch_exactmatch'] = (np.where((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days ==0,
                                                 1,0))

rq1data_text['ActualStartDateMatch_daybefore'] = (np.where(((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days <= 0) &
                                                 ((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-1),
                                                 1,0))

rq1data_text['ActualStartDateMatch_dayeitherside'] = (np.where(((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days <= 1) &
                                                 ((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-1),
                                                 1,0))

rq1data_text['ActualStartDateMatch_weekbefore'] = (np.where(((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days <= 0) &
                                                 ((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-7),
                                                 1,0))

rq1data_text['ActualStartDateMatch_weekeitherside'] = (np.where(((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days <= 7) &
                                                 ((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-7),
                                                 1,0))

rq1data_text['ActualStartDateMatch_twoweekseitherside'] = (np.where(((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days <= 14) &
                                                 ((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-14),
                                                 1,0))
print(rq1data_text['ActualStartDateMatch_exactmatch'].value_counts())
print(rq1data_text['ActualStartDateMatch_daybefore'].value_counts())
print(rq1data_text['ActualStartDateMatch_dayeitherside'].value_counts())
print(rq1data_text['ActualStartDateMatch_weekbefore'].value_counts())
print(rq1data_text['ActualStartDateMatch_weekeitherside'].value_counts())
print(rq1data_text['ActualStartDateMatch_twoweekseitherside'].value_counts())

# Most of the additional matches in day either side come from day before (another 4200, 1 day after adds 30ish)
# Most of the additional matches in week either side come from week before (another 450, 1 week after adds 50ish)
# Additional matches for two weeks either side (another 200)
'''

In [ ]:
import numpy as np
# If the text actual start date is the same or after the structured data actual start date or the referral date minus 1 BUT..
# before the structured data referral close date
# Only take open case up to 3 months 
rq1data_text['BetweenReferralMatch'] = (np.where(((rq1data_text['ActualStartDate_txt'] >= rq1data_text['ReferralDatetime']) & 
                                                 (rq1data_text['ActualStartDate_txt'] <= rq1data_text['ReferralCloseDate'])) |
                                                 ((rq1data_text['ActualStartDate_txt'] >= rq1data_text['ActualStartDate_str']) & 
                                                 (rq1data_text['ActualStartDate_txt'] <= rq1data_text['ReferralCloseDate'])) ,
                                                 1,0))

rq1data_text['BetweenReferralMatch_minus_one_day'] = (np.where((((rq1data_text['ActualStartDate_txt'] - rq1data_text['ReferralDatetime']).dt.days >=-1) & 
                                                 (rq1data_text['ActualStartDate_txt'] <= rq1data_text['ReferralCloseDate'])) |
                                                 (((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-1) & 
                                                 (rq1data_text['ActualStartDate_txt'] <= rq1data_text['ReferralCloseDate'])) ,
                                                 1,0))

rq1data_text['BetweenReferralMatch_minus_one_week'] = (np.where((((rq1data_text['ActualStartDate_txt'] - rq1data_text['ReferralDatetime']).dt.days >=-7) & 
                                                 (rq1data_text['ActualStartDate_txt'] <= rq1data_text['ReferralCloseDate'])) |
                                                 (((rq1data_text['ActualStartDate_txt'] - rq1data_text['ActualStartDate_str']).dt.days >=-7) & 
                                                 (rq1data_text['ActualStartDate_txt'] <= rq1data_text['ReferralCloseDate'])) ,
                                                 1,0))

print(rq1data_text['BetweenReferralMatch'].value_counts())
print(rq1data_text['BetweenReferralMatch_minus_one_day'].value_counts())
print(rq1data_text['BetweenReferralMatch_minus_one_week'].value_counts())

In [ ]:
# Exact match on ActualStartDates seems too strict (only two that are not within the referral period)
# Adding in one day before, adding in one week before only adds another 100ish
# rq1data_text['BetweenReferralMatch_minus_one_day']
#print(pd.crosstab(rq1data_text['ActualStartDateMatch_exactmatch'], rq1data_text['BetweenReferralMatch']))
print(pd.crosstab(rq1data_text['ActualStartDateMatch_dayeitherside'], rq1data_text['BetweenReferralMatch_minus_one_day']))
print(pd.crosstab(rq1data_text['ActualStartDateMatch_dayeitherside'], rq1data_text['BetweenReferralMatch_minus_one_week']))
#print(pd.crosstab(rq1data_text['ActualStartDateMatch_weekeitherside'], rq1data_text['BetweenReferralMatch']))
#print(pd.crosstab(rq1data_text['ActualStartDateMatch_twoweekseitherside'], rq1data_text['BetweenReferralMatch']))

In [ ]:
# 1500 duplicate C / R using 'BetweenReferralMatch_minus_one_day'
# 50 duplicate C / R using 'BetweenReferralMatch'
# 150 duplicate Child Social Work Assessment_text using 'BetweenReferralMatch_minus_one_day'
# 50 duplicate Child Social Work Assessment_text using 'BetweenReferralMatch'
# No observations Child Social Work Assessment to Initial Child Protection Conference_text using 'BetweenReferralMatch' or 'BetweenReferralMatch_minus_one_day' 
# No observations Child Social Work Assessment for Review Child Protection Conference_text using 'BetweenReferralMatch' or 'BetweenReferralMatch_minus_one_day'

In [ ]:
# Select only those which are within the referral period minus one day
# Drop duplicates
print(rq1data_text.shape) 
rq1data_text_matched = rq1data_text.loc[(rq1data_text['BetweenReferralMatch_minus_one_day'] ==1),]
print(rq1data_text_matched.shape)
rq1data_text_matched_de_dup = rq1data_text_matched.drop_duplicates(subset=['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate_str',
       'ActualStartDate_txt', 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'], keep = 'first')
print(rq1data_text_matched_de_dup.shape)

In [ ]:
# Contact / referral forms seem to be the day before CSWA => combine as refers to the same incident
text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'])


rq1data_text_matched_de_dup_concat = rq1data_text_matched_de_dup.groupby(['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate_str'], as_index = False)[text_cols].agg(' '.join)

print(rq1data_text_matched_de_dup.shape)
print(rq1data_text_matched_de_dup_concat.shape)
rq1data_text_matched_de_dup_concat.sort_values(by = ['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate_str'], inplace = True)
rq1data_text_matched_de_dup_concat.head(50)

In [ ]:
# Identify text associated with current referral for rq1
cols_to_drop_not_referral = ['Child Social Work Assessment for Review Child Protection Conference_text',
                            'Child Social Work Assessment to Initial Child Protection Conference_text', 
                            'Child Social Work Assessment_text']

rq1data_text_matched_de_dup_concat.drop(columns = cols_to_drop_not_referral, inplace = True)
rq1data_text_matched_de_dup_concat.head()

#### Rq1data: bring in previous text

In [ ]:
# Merge in just structured data required to select relevant text data
# Don't want all structured data too yet as we'll need to have just text data for extracting text features

rq1data_text_prev = rq1data[['PSID', 'ReferralDatetime_previous', 'ReferralCloseDate_previous', 'ActualStartDate_previous']].merge(df_combined_all_text, how = 'outer', on = 'PSID')
rq1data_text_prev[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text']] = (rq1data_text_prev[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
                                                               'Child Social Work Assessment to Initial Child Protection Conference_text',
                                                               'Child Social Work Assessment_text', 'Contact and Referral Form_text']].fillna(''))

(rq1data_text_prev.rename(columns = {'ActualStartDate':'ActualStartDate_txt_prev',
                                 'ActualStartDate_previous':'ActualStartDate_str_prev',
                                'ReferralDatetime_previous':'ReferralDatetime_str_prev',
                                 'ReferralCloseDate_previous':'ReferralCloseDate_str_prev'},
                                inplace = True))

rq1data_text_prev['ActualStartDate_str_prev'] = pd.to_datetime(rq1data_text_prev['ActualStartDate_str_prev'], format = "%Y-%m-%d")
rq1data_text_prev['ReferralDatetime_str_prev'] = pd.to_datetime(rq1data_text_prev['ReferralDatetime_str_prev'], format = "%Y-%m-%d")
rq1data_text_prev['ReferralCloseDate_str_prev'] = pd.to_datetime(rq1data_text_prev['ReferralCloseDate_str_prev'], format = "%Y-%m-%d")

# text ActualStartDate is before structured data ActualStartDate and ReferralDatetime (minus 1 day)
rq1data_text_prev['BeforeReferralMatch_minus_one_day_prev'] = (np.where((rq1data_text_prev['ActualStartDate_txt_prev'] < (rq1data_text_prev['ReferralDatetime_str_prev']-pd.Timedelta(days = 1))) |
                                                 (rq1data_text_prev['ActualStartDate_txt_prev'] < (rq1data_text_prev['ActualStartDate_str_prev']-pd.Timedelta(days = 1))) ,
                                                 1,0))

rq1data_text_prev['BeforeReferralMatch_minus_one_day_prev'].value_counts()
#rq1data_text_prev.loc[rq1data_text_prev['BeforeReferralMatch_minus_one_day_prev']==1,['ActualStartDate_txt_prev', 'ReferralDatetime_str_prev','ActualStartDate_str_prev']]

In [ ]:
# Select only those which are within the referral period minus one day
# Drop duplicates
print(rq1data_text_prev.shape) 
rq1data_text_prev_matched = rq1data_text_prev.loc[(rq1data_text_prev['BeforeReferralMatch_minus_one_day_prev'] ==1),]
print(rq1data_text_prev_matched.shape)
rq1data_text_prev_matched_de_dup = rq1data_text_prev_matched.drop_duplicates(subset=['PSID', 'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
       'ActualStartDate_str_prev','ActualStartDate_txt_prev', 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'], keep = 'first')
print(rq1data_text_prev_matched_de_dup.shape)


In [ ]:
# Contact / referral forms seem to be the day before CSWA => combine as refers to the same incident
text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'])

rq1data_text_prev_matched_de_dup_concat = rq1data_text_prev_matched_de_dup.groupby(['PSID', 'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
       'ActualStartDate_str_prev'], as_index = False)[text_cols].agg(' '.join)

print(rq1data_text_prev_matched_de_dup.shape)
print(rq1data_text_prev_matched_de_dup_concat.shape)
(rq1data_text_prev_matched_de_dup_concat.sort_values(by = ['PSID', 'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
       'ActualStartDate_str_prev'], inplace = True))
rq1data_text_prev_matched_de_dup_concat.head(50)

#### Rq1 - merge together current and previous text

In [ ]:
rq1data['ReferralDatetime'] = pd.to_datetime(rq1data['ReferralDatetime'], format = "%Y-%m-%d")
rq1data[ 'ReferralDatetime_previous'] = pd.to_datetime(rq1data['ReferralDatetime_previous'], format = "%Y-%m-%d")
rq1data_for_merging = (rq1data[['PSID', 'ReferralDatetime', 'ReferralDatetime_previous']]
        .merge(rq1data_text_matched_de_dup_concat, how = 'left', on = ['PSID', 'ReferralDatetime']))

rename_prev = {'Child Social Work Assessment for Review Child Protection Conference_text': 
               'Child Social Work Assessment for Review Child Protection Conference_text_prev',
               'Child Social Work Assessment to Initial Child Protection Conference_text':
               'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
               'Child Social Work Assessment_text': 'Child Social Work Assessment_text_prev', 
               'Contact and Referral Form_text': 'Contact and Referral Form_text_prev'}

rq1data_text_prev_matched_de_dup_concat.rename(columns = rename_prev, inplace = True)

rq1data_text_for_anonymisation = (rq1data_for_merging.merge(rq1data_text_prev_matched_de_dup_concat, how = 'left', 
                                                 left_on = ['PSID', 'ReferralDatetime_previous'], right_on =  ['PSID', 'ReferralDatetime_str_prev']))

In [ ]:
# 'Contact and Referral Form_text' and 'Contact and Referral Form_text_prev' are different except a small no. of cases
# Most probably just copied across by social worker
#rq1data_text_for_anonymisation.sort_values(by = ['PSID', 'ReferralDatetime'], inplace = True)
#rq1data_text_for_anonymisation[['PSID', 'ReferralDatetime', 'ReferralDatetime_previous', 'Contact and Referral Form_text_prev', 'Contact and Referral Form_text']].head(50)
(rq1data_text_for_anonymisation['Contact and Referral Form_text_prev'] == rq1data_text_for_anonymisation['Contact and Referral Form_text']).value_counts() 

In [ ]:
# Selecting just text columns and merging keys
date_cols_for_selecting = (['ReferralCloseDate', 'ActualStartDate_str',
        'ReferralDatetime_str_prev','ReferralCloseDate_str_prev', 'ActualStartDate_str_prev'])

rq1data_text_for_anonymisation.drop(columns = date_cols_for_selecting, inplace = True)

print(rq1data_text_for_anonymisation.columns)

In [ ]:
# Number of Child Social Work Assessment for Review Child Protection Conference_text_prev and
# Child Social Work Assessment to Initial Child Protection Conference_text_prev is better with improved way of 
# merging previous
for col in rq1data_text_for_anonymisation.columns:
    print(col)
    print(rq1data_text_for_anonymisation[col].nunique())

In [ ]:
# Pickle final text data to feed into anonymisation
import pickle
with open("Created\\rq1data_text_for_anonymisation.pkl", "wb") as handle:
    pickle.dump(rq1data_text_for_anonymisation, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Combine with rq2data

In [ ]:
assert 1==2

In [ ]:
import os
import pandas as pd
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename = open("Created\\df_combined_all_text.pkl", "rb")
df_combined_all_text = pickle.load(filename)
print(df_combined_all_text.shape)

rq2data = pd.read_csv("..\\Updated Structured Data\\Created\\rq2data_w_previous.csv", index_col = 0)
print(rq2data.shape)
#print(rq2data.index)
#print(rq2data.columns)

In [ ]:
# Merge in just structured data required to select relevant text data
# Don't want all structured data too yet as we'll need to have just text data for extracting text features

rq2data_text = rq2data[['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate', 'AssessmentType']].merge(df_combined_all_text, how = 'outer', on = 'PSID')
rq2data_text[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text']] = (rq2data_text[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
                                                               'Child Social Work Assessment to Initial Child Protection Conference_text',
                                                               'Child Social Work Assessment_text', 'Contact and Referral Form_text']].fillna(''))
(rq2data_text.rename(columns = {'ActualStartDate_x':'ActualStartDate_str',
                                 'ActualStartDate_y':'ActualStartDate_txt'},
                                inplace = True))

rq2data_text['ActualStartDate_str'] = pd.to_datetime(rq2data_text['ActualStartDate_str'], format = "%Y-%m-%d")
rq2data_text['ReferralDatetime'] = pd.to_datetime(rq2data_text['ReferralDatetime'], format = "%Y-%m-%d")
rq2data_text['ReferralCloseDate'] = pd.to_datetime(rq2data_text['ReferralCloseDate'], format = "%Y-%m-%d")

In [ ]:
import numpy as np
# If the text actual start date is the same or after the structured data actual start date or the referral date minus 1 BUT..
# within 2 weeks of the referral 
# Checks indicated the same pattern as for rq1
# Only include text within first 2 weeks of referral if predicting escalation 3 months out

rq2data_text['Twoweekssafterref'] = rq2data_text['ReferralDatetime'] + pd.Timedelta(days = 14)

rq2data_text['Match_minus_one_day_Twoweekssafterass'] = (np.where((((rq2data_text['ActualStartDate_txt'] - rq2data_text['ReferralDatetime']).dt.days >=-1) & 
                                                 (rq2data_text['ActualStartDate_txt'] <= rq2data_text['Twoweekssafterref'])) |
                                                 (((rq2data_text['ActualStartDate_txt'] - rq2data_text['ActualStartDate_str']).dt.days >=-1) & 
                                                 (rq2data_text['ActualStartDate_txt'] <= rq2data_text['Twoweekssafterref'])) ,
                                                 1,0))

print(rq2data_text['Match_minus_one_day_Twoweekssafterass'].value_counts())

In [ ]:
# Select only those which are within two weeks of the referral minus one day
# Drop duplicates
print(rq2data_text.shape) 
rq2data_text_matched = rq2data_text.loc[(rq2data_text['Match_minus_one_day_Twoweekssafterass'] ==1),]
print(rq2data_text_matched.shape)
rq2data_text_matched_de_dup = rq2data_text_matched.drop_duplicates(subset=['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate_str',
       'ActualStartDate_txt', 'AssessmentType', 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'], keep = 'first')
print(rq2data_text_matched_de_dup.shape)

In [ ]:
# Contact / referral forms seem to be the day before CSWA => combine as refers to the same incident
text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'])


rq2data_text_matched_de_dup_concat = rq2data_text_matched_de_dup.groupby(['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate_str', 'AssessmentType'], as_index = False)[text_cols].agg(' '.join)

print(rq2data_text_matched_de_dup.shape)
print(rq2data_text_matched_de_dup_concat.shape)
rq2data_text_matched_de_dup_concat.sort_values(by = ['PSID', 'ReferralDatetime', 'ReferralCloseDate', 'ActualStartDate_str'], inplace = True)
rq2data_text_matched_de_dup_concat.head(50)

#### Rq2data: bring in previous text

In [ ]:
# Merge in just structured data required to select relevant text data
# Don't want all structured data too yet as we'll need to have just text data for extracting text features

rq2data_cols_for_merge = (['PSID', 'ReferralDatetime_previous', 'ReferralCloseDate_previous', 'ActualStartDate_previous', 
                            'previous_count_AssessmentType_ Child Social Work Assessment',
 'previous_count_AssessmentType_ Child Social Work Assessment for Review Child Protection Conference',
 'previous_count_AssessmentType_ Child Social Work Assessment to Initial Child Protection Conference'])


rq2data_text_prev = rq2data[rq2data_cols_for_merge].merge(df_combined_all_text, how = 'outer', on = 'PSID')
rq2data_text_prev[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text']] = (rq2data_text_prev[[ 'Child Social Work Assessment for Review Child Protection Conference_text',
                                                               'Child Social Work Assessment to Initial Child Protection Conference_text',
                                                               'Child Social Work Assessment_text', 'Contact and Referral Form_text']].fillna(''))

(rq2data_text_prev.rename(columns = {'ActualStartDate':'ActualStartDate_txt_prev',
                                 'ActualStartDate_previous':'ActualStartDate_str_prev',
                                'ReferralDatetime_previous':'ReferralDatetime_str_prev',
                                 'ReferralCloseDate_previous':'ReferralCloseDate_str_prev'},
                                inplace = True))

rq2data_text_prev['ActualStartDate_str_prev'] = pd.to_datetime(rq2data_text_prev['ActualStartDate_str_prev'], format = "%Y-%m-%d")
rq2data_text_prev['ReferralDatetime_str_prev'] = pd.to_datetime(rq2data_text_prev['ReferralDatetime_str_prev'], format = "%Y-%m-%d")
rq2data_text_prev['ReferralCloseDate_str_prev'] = pd.to_datetime(rq2data_text_prev['ReferralCloseDate_str_prev'], format = "%Y-%m-%d")


rq2data_text_prev['BeforeReferralMatch_minus_one_day_prev'] = (np.where((rq2data_text_prev['ActualStartDate_txt_prev'] < (rq2data_text_prev['ReferralDatetime_str_prev']-pd.Timedelta(days = 1))) |
                                                 (rq2data_text_prev['ActualStartDate_txt_prev'] < (rq2data_text_prev['ActualStartDate_str_prev']-pd.Timedelta(days = 1))) ,
                                                 1,0))

In [ ]:
# Select only those which are within the referral period minus one day
# Drop duplicates
print(rq2data_text_prev.shape) 
rq2data_text_prev_matched = rq2data_text_prev.loc[(rq2data_text_prev['BeforeReferralMatch_minus_one_day_prev'] ==1),]
print(rq2data_text_prev_matched.shape)
rq2data_text_prev_matched_de_dup = rq2data_text_prev_matched.drop_duplicates(subset=['PSID', 'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
       'ActualStartDate_str_prev','ActualStartDate_txt_prev', 'previous_count_AssessmentType_ Child Social Work Assessment',
 'previous_count_AssessmentType_ Child Social Work Assessment for Review Child Protection Conference',
 'previous_count_AssessmentType_ Child Social Work Assessment to Initial Child Protection Conference',
  'Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'], keep = 'first')
print(rq2data_text_prev_matched_de_dup.shape)

In [ ]:
# Contact / referral forms seem to be the day before CSWA => combine as refers to the same incident
text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text'])

rq2data_text_prev_matched_de_dup_concat = rq2data_text_prev_matched_de_dup.groupby(['PSID', 'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
       'ActualStartDate_str_prev', 'previous_count_AssessmentType_ Child Social Work Assessment',
       'previous_count_AssessmentType_ Child Social Work Assessment for Review Child Protection Conference',
       'previous_count_AssessmentType_ Child Social Work Assessment to Initial Child Protection Conference'], as_index = False)[text_cols].agg(' '.join)

print(rq2data_text_prev_matched_de_dup.shape)
print(rq2data_text_prev_matched_de_dup_concat.shape)
(rq2data_text_prev_matched_de_dup_concat.sort_values(by = ['PSID', 'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
       'ActualStartDate_str_prev'], inplace = True))
rq2data_text_prev_matched_de_dup_concat.head(50)

#### Rq2 - merge together current and previous text

In [ ]:
rq2data['ReferralDatetime'] = pd.to_datetime(rq2data['ReferralDatetime'], format = "%Y-%m-%d")
rq2data_for_merging = (rq2data[['PSID', 'ReferralDatetime', 'ReferralDatetime_previous']]
        .merge(rq2data_text_matched_de_dup_concat, how = 'left', on = ['PSID', 'ReferralDatetime']))

rename_prev = {'Child Social Work Assessment for Review Child Protection Conference_text': 
               'Child Social Work Assessment for Review Child Protection Conference_text_prev',
               'Child Social Work Assessment to Initial Child Protection Conference_text':
               'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
               'Child Social Work Assessment_text': 'Child Social Work Assessment_text_prev', 
               'Contact and Referral Form_text': 'Contact and Referral Form_text_prev'}

rq2data_text_prev_matched_de_dup_concat.rename(columns = rename_prev, inplace = True)

rq2data_text_for_anonymisation = (rq2data_for_merging.merge(rq2data_text_prev_matched_de_dup_concat, how = 'left', 
                                                 left_on = ['PSID', 'ReferralDatetime'], right_on =  ['PSID', 'ReferralDatetime_str_prev']))

In [ ]:
# Selecting just text columns and merging keys
date_cols_for_selecting = (['ReferralDatetime_previous','ReferralCloseDate', 'ActualStartDate_str',
                            'ReferralDatetime_str_prev', 'ReferralCloseDate_str_prev',
                           'ActualStartDate_str_prev'])

assessments_cols_for_selecting = (['AssessmentType', 'previous_count_AssessmentType_ Child Social Work Assessment',
       'previous_count_AssessmentType_ Child Social Work Assessment for Review Child Protection Conference',
       'previous_count_AssessmentType_ Child Social Work Assessment to Initial Child Protection Conference'])

cols_for_selecting = date_cols_for_selecting + assessments_cols_for_selecting

rq2data_text_for_anonymisation.drop(columns = cols_for_selecting, inplace = True)

print(rq2data_text_for_anonymisation.columns)

In [ ]:
for col in rq2data_text_for_anonymisation.columns:
    print(col)
    print(rq2data_text_for_anonymisation[col].nunique())

In [ ]:
# Pickle final text data to feed into anonymisation
import pickle

with open("Created\\rq2data_text_for_anonymisation.pkl", "wb") as handle:
    pickle.dump(rq2data_text_for_anonymisation, handle, protocol = pickle.HIGHEST_PROTOCOL)